In [1]:
# General libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
# Machine Learning Libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier 

## Preliminary work on the dataset

In [2]:
yahoo_df = pd.read_csv("../data/NSQ.csv")
yahoo_df = yahoo_df.set_index("Date")

X = yahoo_df[['ROC_30', '4 Day ROI', 'EMA_12', 'MACD_12_26_9', 'SRSI_30',
       'Williams_14', 'ATR_14', 'Previous_differenced']]
y = yahoo_df["Up down"]

# Split in 80/20 the dataframeX
X_train_80, X_test, y_train_80, y_test = train_test_split(X, y, test_size = 0.20,
                                                          shuffle=False)

# Split in 75/25 the remaining 80 %
X_train, X_valid, y_train, y_valid = train_test_split(X_train_80, y_train_80,
                                                      test_size = 0.25, shuffle=False)


# Normalize

In [3]:
def shift_norm(df):
    #df = data
    for column in df:
        df[column]=((df[column]-df[column].mean())/df[column].std())
    return df

In [4]:
X_train_80 = shift_norm(X_train_80)
X_test = shift_norm(X_test)

In [5]:
X_train = shift_norm(X_train)
X_valid = shift_norm(X_valid)

# kNN

In [6]:
# Train and fit
kNN = KNeighborsClassifier()
kNN.fit(X_train, y_train)

# Predict
y_pred_train = kNN.predict(X_train)
y_pred_valid = kNN.predict(X_valid)

# Compute Accuracy
train_acc = accuracy_score(y_train,y_pred_train)
valid_acc = accuracy_score(y_valid, y_pred_valid)

print("Train Accuracy: {:.5f} - Validation Accuracy: {:.5f}".format(train_acc, valid_acc))

Train Accuracy: 0.70384 - Validation Accuracy: 0.50662


### Tuning parameters

In [7]:
knn = KNeighborsClassifier()

k_range = list(range(1,31))
weight_options = ["uniform", "distance"]
param_grid = dict(n_neighbors = k_range, weights = weight_options)
  
# defining parameter range
grid = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy')
  
# fitting the model for grid search
grid_search=grid.fit(X_train, y_train)

In [8]:
print(grid_search.best_estimator_)

KNeighborsClassifier(n_neighbors=21)


In [9]:
# Train and fit
kNN = KNeighborsClassifier(n_neighbors=2)
kNN.fit(X_train_80, y_train_80)

# Predict
y_pred_test = kNN.predict(X_test)
y_prob = kNN.predict_proba(X_test)[:,1]

# Finally evaluate on test
test_acc = accuracy_score(y_test, y_pred_test)
print ("Test Accuracy: {:.5f}".format(test_acc) )

# Store the Test Accuracy 
kNN_accuracy_test = test_acc

Test Accuracy: 0.48299


# Save data prediction

In [10]:
df_prediction = pd.read_csv("../data/model_accuracy_nsq.csv", index_col = False)
df_prediction["y"] = y_test.values
df_prediction["kNN_pred"] = y_pred_test
df_prediction["kNN_accuracy"] = [kNN_accuracy_test for x in range(len(y_pred_test))]
df_prediction["kNN_prob"] = y_prob

In [11]:
df_prediction = df_prediction.drop(columns=["Unnamed: 0"])
df_prediction

,RF_pred,RF_accuracy,RF_prob,SVC_pred,SVC_accuracy,SVC_prob,XGB_pred,XGB_accuracy,XGB_prob,ANN_pred,ANN_accuracy,ANN_prob,LGBM_pred,LGBM_accuracy,LGBM_prob,y,kNN_pred,kNN_accuracy,kNN_prob
0,1,0.523629,0.657543,1,0.558601,0.544082,1,0.533081,0.536098,0,0.441399,0.464561,1,0.529301,0.532322,0,0,0.482987,0.5
1,0,0.523629,0.324182,1,0.558601,0.544076,0,0.533081,0.390463,0,0.441399,0.464505,1,0.529301,0.518503,1,0,0.482987,0.5
2,0,0.523629,0.496864,1,0.558601,0.545460,1,0.533081,0.552227,0,0.441399,0.464505,1,0.529301,0.515129,1,1,0.482987,1.0
3,1,0.523629,0.574333,1,0.558601,0.545009,1,0.533081,0.596263,0,0.441399,0.464505,1,0.529301,0.531008,1,0,0.482987,0.5
4,0,0.523629,0.421569,1,0.558601,0.543872,0,0.533081,0.483318,0,0.441399,0.464505,1,0.529301,0.529314,0,0,0.482987,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1053,1,0.523629,0.679418,1,0.558601,0.539406,1,0.533081,0.598255,0,0.441399,0.464011,1,0.529301,0.510938,0,1,0.482987,1.0
1054,1,0.523629,0.555801,1,0.558601,0.537904,1,0.533081,0.567730,0,0.441399,0.464421,1,0.529301,0.511045,1,0,0.482987,0.5
1055,1,0.523629,0.573323,1,0.558601,0.540966,1,0.533081,0.561254,0,0.441399,0.463568,1,0.529301,0.509250,0,0,0.482987,0.0
1056,1,0.523629,0.719503,1,0.558601,0.536726,1,0.533081,0.590790,0,0.441399,0.464013,1,0.529301,0.510938,1,1,0.482987,1.0


In [12]:
df_prediction.to_csv("../data/model_accuracy_nsq.csv")